### 1. B-Spline 기저 함수

In [1]:
import torch

def B_batch(x, grid, k=0, device='cpu'):
    '''
    주어진 점 x에서 B-스플라인 기저 함수를 평가합니다.

    Args:
    -----
        x : torch.Tensor
            입력 텐서로, 크기는 (batch_size, in_dim).
        grid : torch.Tensor
            노드 벡터로, 크기는 (in_dim, num_knots).
        k : int
            B-스플라인의 차수.
        device : str
            연산을 수행할 디바이스.

    Returns:
    --------
        value : torch.Tensor
            평가된 B-스플라인 기저 함수로, 크기는 (batch_size, in_dim, num_basis).
    '''
    x = x.unsqueeze(dim=2)
    grid = grid.unsqueeze(dim=0)

    if k == 0:
        value = ((x >= grid[:, :, :-1]) & (x < grid[:, :, 1:])).float()
    else:
        B_km1 = B_batch(x[:, :, 0], grid[0], k=k - 1).to(device)
        denom1 = grid[:, :, k:-1] - grid[:, :, :-(k + 1)]
        denom2 = grid[:, :, k + 1:] - grid[:, :, 1:(-k)]
        denom1[denom1 == 0] = 1e-8
        denom2[denom2 == 0] = 1e-8
        term1 = ((x - grid[:, :, :-(k + 1)]) / denom1) * B_km1[:, :, :-1]
        term2 = ((grid[:, :, k + 1:] - x) / denom2) * B_km1[:, :, 1:]
        value = term1 + term2

    value = torch.nan_to_num(value)
    return value

In [2]:
# 입력 데이터
x = torch.linspace(-1, 1, steps=5).unsqueeze(1)  # (batch_size, in_dim)

# 노드 벡터 생성
grid = torch.linspace(-1, 1, steps=5).unsqueeze(0)  # (in_dim, num_knots)

# 차수 설정
k = 2

# 함수 호출
B_values = B_batch(x, grid, k=k)

print("B-스플라인 기저 함수 값:")
print(B_values)

B-스플라인 기저 함수 값:
tensor([[[0.0000, 0.0000]],

        [[0.5000, 0.0000]],

        [[0.5000, 0.5000]],

        [[0.0000, 0.5000]],

        [[0.0000, 0.0000]]])


### 2. 계수와 곡선 간의 변환 함수

In [3]:
def coef2curve(x_eval, grid, coef, k, device='cpu'):
    '''
    주어진 계수를 사용하여 x_eval에서 B-스플라인 곡선을 평가합니다.

    Args:
    -----
        x_eval : torch.Tensor
            크기 (batch_size, in_dim)의 입력 텐서.
        grid : torch.Tensor
            크기 (in_dim, num_knots)의 노드 벡터.
        coef : torch.Tensor
            크기 (in_dim, out_dim, num_basis)의 B-스플라인 계수.
        k : int
            B-스플라인의 차수.
        device : str
            연산을 수행할 디바이스.

    Returns:
    --------
        y_eval : torch.Tensor
            크기 (batch_size, in_dim, out_dim)의 평가된 스플라인 곡선.
    '''
    b_splines = B_batch(x_eval, grid, k=k, device=device)
    y_eval = torch.einsum('ijk,jlk->ijl', b_splines, coef.to(device))
    return y_eval

def curve2coef(x_eval, y_eval, grid, k, lamb=1e-8):
    '''
    주어진 곡선 평가값으로부터 B-스플라인 계수를 계산합니다.

    Args:
    -----
        x_eval : torch.Tensor
            크기 (batch_size, in_dim)의 입력 텐서.
        y_eval : torch.Tensor
            크기 (batch_size, in_dim, out_dim)의 평가된 곡선 값.
        grid : torch.Tensor
            크기 (in_dim, num_knots)의 노드 벡터.
        k : int
            B-스플라인의 차수.
        lamb : float
            정규화 파라미터.

    Returns:
    --------
        coef : torch.Tensor
            크기 (in_dim, out_dim, num_basis)의 계산된 B-스플라인 계수.
    '''
    batch = x_eval.shape[0]
    in_dim = x_eval.shape[1]
    out_dim = y_eval.shape[2]
    n_coef = grid.shape[1] - k - 1

    mat = B_batch(x_eval, grid, k).permute(1, 0, 2)
    mat = mat.unsqueeze(1).expand(-1, out_dim, -1, -1)
    y_eval = y_eval.permute(1, 2, 0).unsqueeze(3)

    XtX = torch.einsum('ijmn,ijnp->ijmp', mat.transpose(2, 3), mat)
    Xty = torch.einsum('ijmn,ijnp->ijmp', mat.transpose(2, 3), y_eval)
    n1, n2, n = XtX.shape[0], XtX.shape[1], XtX.shape[2]
    identity = torch.eye(n, n, device=mat.device).unsqueeze(0).unsqueeze(0).expand(n1, n2, -1, -1)
    A = XtX + lamb * identity
    B = Xty
    coef = torch.linalg.solve(A, B).squeeze(-1)
    return coef

In [4]:
# 계수와 곡선 간의 변환 확인

# 입력 데이터 생성
x_eval = torch.linspace(-1, 1, steps=10).unsqueeze(1)  # (batch_size, in_dim)
y_true = torch.sin(x_eval)  # 실제 곡선 값

# 노드 벡터 생성
grid = torch.linspace(-1, 1, steps=5).unsqueeze(0)  # (in_dim, num_knots)
k = 3  # 차수

# 곡선으로부터 계수 계산
coef = curve2coef(x_eval, y_true.unsqueeze(2), grid, k)

# 계수를 사용하여 곡선 복원
y_pred = coef2curve(x_eval, grid, coef, k)

print("원래 곡선 값:")
print(y_true.squeeze())
print("복원된 곡선 값:")
print(y_pred.squeeze())

원래 곡선 값:
tensor([-0.8415, -0.7017, -0.5274, -0.3272, -0.1109,  0.1109,  0.3272,  0.5274,
         0.7017,  0.8415])
복원된 곡선 값:
tensor([0.0000e+00, 2.4021e-10, 1.9217e-09, 6.0803e-09, 1.0224e-08, 1.0224e-08,
        6.0803e-09, 1.9217e-09, 2.4021e-10, 0.0000e+00])


### 3. 그리드 확장

In [5]:
def extend_grid(grid, k_extend=0):
    '''
    스플라인의 경계 조건을 처리하기 위해 그리드를 확장합니다.

    Args:
    -----
        grid : torch.Tensor
            크기 (in_dim, num_knots)의 원래 노드 벡터.
        k_extend : int
            각 경계에서 확장할 노드의 수.

    Returns:
    --------
        extended_grid : torch.Tensor
            확장된 노드 벡터.
    '''
    h = (grid[:, -1:] - grid[:, :1]) / (grid.shape[1] - 1)
    for _ in range(k_extend):
        grid = torch.cat([grid[:, :1] - h, grid, grid[:, -1:] + h], dim=1)
    return grid

In [6]:
# 그리드 확장 테스트

# 원래 그리드 생성
grid = torch.linspace(0, 1, steps=5).unsqueeze(0)  # (in_dim, num_knots)

# 그리드 확장
k_extend = 2  # 확장할 노드 수
extended_grid = extend_grid(grid, k_extend=k_extend)

print("원래 그리드:")
print(grid)
print("확장된 그리드:")
print(extended_grid)


원래 그리드:
tensor([[0.0000, 0.2500, 0.5000, 0.7500, 1.0000]])
확장된 그리드:
tensor([[-0.5000, -0.2500,  0.0000,  0.2500,  0.5000,  0.7500,  1.0000,  1.2500,
          1.5000]])


### 4. KAN Layer

In [7]:
import torch
import torch.nn as nn
import numpy as np

class KANLayer(nn.Module):
    """
    KANLayer 클래스

    Attributes:
    -----------
        in_dim : int
            입력 차원.
        out_dim : int
            출력 차원.
        num : int
            그리드 구간 수.
        k : int
            B-스플라인의 차수.
        noise_scale : float
            초기화 시 노이즈의 스케일.
        coef : torch.Tensor
            B-스플라인 기저 함수의 계수.
        scale_base_mu : float
            기본 함수 스케일링 계수의 평균.
        scale_base_sigma : float
            기본 함수 스케일링 계수의 표준편차.
        scale_sp : float
            스플라인 함수의 스케일링 계수.
        base_fun : callable
            기본 함수 b(x).
        mask : torch.Tensor
            활성/비활성 활성화를 나타내는 마스크.
        grid_eps : float
            그리드 적응을 제어하는 파라미터.
        device : str
            연산을 수행할 디바이스.
    """

    def __init__(self, in_dim=3, out_dim=2, num=5, k=3, noise_scale=0.1,
                 scale_base_mu=0.0, scale_base_sigma=1.0, scale_sp=1.0,
                 base_fun=torch.nn.SiLU(), grid_eps=0.02, grid_range=[-1, 1],
                 sp_trainable=True, sb_trainable=True, device='cpu', sparse_init=False):
        super(KANLayer, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.num = num
        self.k = k
        self.grid_eps = grid_eps
        self.device = device

        # 그리드 초기화
        grid = torch.linspace(grid_range[0], grid_range[1], steps=num + 1).unsqueeze(0).repeat(in_dim, 1)
        grid = extend_grid(grid, k_extend=k)  # (in_dim, num + 1 + 2k)
        self.grid = nn.Parameter(grid, requires_grad=False)

        # 계수 초기화
        num_knots = num + 1 + 2 * k  # 그리드 점의 총 수
        num_basis = num + k  # B-스플라인 기저 함수의 수

        # x_eval의 차원: (num + 1, in_dim)
        x_eval = self.grid[:, k:-k].t()  # 내부 그리드 점들

        # noises의 차원을 x_eval과 맞춤: (num + 1, in_dim, out_dim)
        noises = (torch.rand(num + 1, in_dim, out_dim) - 0.5) * noise_scale / num

        # coef 계산
        self.coef = nn.Parameter(curve2coef(x_eval, noises, self.grid, k), requires_grad=True)

        # 나머지 초기화 코드는 그대로 유지
        self.scale_base = nn.Parameter(
            scale_base_mu / np.sqrt(in_dim) +
            scale_base_sigma * (torch.rand(in_dim, out_dim) * 2 - 1) / np.sqrt(in_dim),
            requires_grad=sb_trainable
        )
        self.scale_sp = nn.Parameter(torch.ones(in_dim, out_dim) * scale_sp, requires_grad=sp_trainable)

        self.base_fun = base_fun

        if sparse_init:
            self.mask = nn.Parameter(torch.randint(0, 2, (in_dim, out_dim)).float(), requires_grad=False)
        else:
            self.mask = nn.Parameter(torch.ones(in_dim, out_dim), requires_grad=False)

        self.to(device)

    def forward(self, x):
        batch_size = x.shape[0]
        preacts = x.unsqueeze(1).expand(batch_size, self.out_dim, self.in_dim)
        base = self.base_fun(x)
        y_spline = coef2curve(x, self.grid, self.coef, self.k, device=self.device)
        postspline = y_spline.permute(0, 2, 1)
        y = self.scale_base.unsqueeze(0) * base.unsqueeze(2) + self.scale_sp.unsqueeze(0) * y_spline
        y = self.mask.unsqueeze(0) * y
        postacts = y.permute(0, 2, 1)
        y = y.sum(dim=1)
        return y, preacts, postacts, postspline


In [8]:
# KANLayer 사용 예시

# 입력 데이터 생성
x = torch.randn(10, 3)  # (batch_size, in_dim)

# KANLayer 초기화
kan_layer = KANLayer(in_dim=3, out_dim=2, num=5, k=3)

# 순전파 실행
y, preacts, postacts, postspline = kan_layer(x)

print("출력 y:")
print(y)
print("활성화 함수 전의 값 preacts:")
print(preacts)
print("활성화 함수 후의 값 postacts:")
print(postacts)
print("스플라인 함수 출력 postspline:")
print(postspline)


출력 y:
tensor([[-0.0908,  0.1939],
        [ 0.6888, -0.4251],
        [ 0.9379, -0.7289],
        [ 0.1921, -0.0386],
        [ 0.1951,  0.0141],
        [ 0.5350, -0.3471],
        [-0.0432, -0.2205],
        [ 0.5574, -0.2622],
        [ 0.3228, -0.2377],
        [-0.1825, -0.0668]], grad_fn=<SumBackward1>)
활성화 함수 전의 값 preacts:
tensor([[[-1.8268, -0.4919, -0.5655],
         [-1.8268, -0.4919, -0.5655]],

        [[-0.8122, -0.0561,  2.9039],
         [-0.8122, -0.0561,  2.9039]],

        [[ 2.1257, -0.1937,  1.4044],
         [ 2.1257, -0.1937,  1.4044]],

        [[-0.0954, -0.4374,  0.8750],
         [-0.0954, -0.4374,  0.8750]],

        [[ 0.5969, -1.4982, -0.0124],
         [ 0.5969, -1.4982, -0.0124]],

        [[ 0.4736, -0.1001,  1.8327],
         [ 0.4736, -0.1001,  1.8327]],

        [[-1.0392,  0.6603,  0.8547],
         [-1.0392,  0.6603,  0.8547]],

        [[ 0.2772, -0.5318,  1.8793],
         [ 0.2772, -0.5318,  1.8793]],

        [[ 1.4182, -0.2921, -1.3811],
      

### 5. Symbolic KAN Layer

In [ ]:
import torch
import torch.nn as nn
import sympy as sp

# 기호적 함수 라이브러리
SYMBOLIC_LIB = {
    'sin': (torch.sin, sp.sin, 'sin'),
    'cos': (torch.cos, sp.cos, 'cos'),
    'exp': (torch.exp, sp.exp, 'exp'),
    'tanh': (torch.tanh, sp.tanh, 'tanh')
}

class Symbolic_KANLayer(nn.Module):
    def __init__(self, in_dim=3, out_dim=2, device='cpu'):
        super(Symbolic_KANLayer, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.device = device

        # 마스크 초기화: 모든 연결을 비활성화
        self.mask = nn.Parameter(torch.zeros(out_dim, in_dim, device=device), requires_grad=False)
        
        # 기호적 함수 리스트 초기화
        self.funs = [[None for _ in range(in_dim)] for _ in range(out_dim)]
        self.funs_name = [['0' for _ in range(in_dim)] for _ in range(out_dim)]
        self.affine = nn.Parameter(torch.ones(out_dim, in_dim, 4, device=device), requires_grad=True)  # [a, b, c, d]

        self.to(device)

    def forward(self, x, singularity_avoiding=False, y_th=10.):
        batch_size = x.shape[0]
        postacts = []

        for j in range(self.out_dim):
            neuron_output = torch.zeros(batch_size, device=self.device)
            for i in range(self.in_dim):
                if self.mask[j, i] > 0 and self.funs[j][i] is not None:
                    a, b, c, d = self.affine[j, i]
                    transformed_x = a * x[:, i] + b
                    func_output = self.funs[j][i](transformed_x)
                    if singularity_avoiding and 'tan' in self.funs_name[j][i]:
                        # 예시로 tan 함수에 대한 특이점 회피 처리
                        func_output = func_output / (1 + (func_output / y_th).abs())
                    neuron_output += c * func_output + d
            postacts.append(neuron_output.unsqueeze(1))

        postacts = torch.cat(postacts, dim=1)  # (batch_size, out_dim)
        y = postacts
        return y, postacts

    def fix_symbolic(self, j, i, fun_name, a_init=None, b_init=None, c_init=None, d_init=None, verbose=True):
        '''
        특정 출력 뉴런 j와 입력 뉴런 i에 기호적 함수 할당 및 아핀 파라미터 초기화

        Args:
        -----
            j : int
                출력 뉴런 인덱스.
            i : int
                입력 뉴런 인덱스.
            fun_name : str
                할당할 기호적 함수 이름 (SYMBOLIC_LIB에 정의된 것).
            a_init, b_init, c_init, d_init : float or None
                아핀 변환 파라미터 초기값. None이면 기본값 사용.
            verbose : bool
                함수 할당 시 출력 여부.
        '''
        if fun_name in SYMBOLIC_LIB:
            fun, fun_sympy, fun_str = SYMBOLIC_LIB[fun_name]
            self.funs[j][i] = fun
            self.funs_name[j][i] = fun_str

            # 아핀 파라미터 초기화
            with torch.no_grad():
                self.affine[j, i, 0] = a_init if a_init is not None else 1.0
                self.affine[j, i, 1] = b_init if b_init is not None else 0.0
                self.affine[j, i, 2] = c_init if c_init is not None else 1.0
                self.affine[j, i, 3] = d_init if d_init is not None else 0.0

            # 마스크 활성화
            with torch.no_grad():
                self.mask[j, i] = 1.0

            if verbose:
                print(f"Assigned function '{fun_name}' to Layer, Neuron {j}, Input {i}")
        else:
            if verbose:
                print(f"Symbolic function '{fun_name}' is not defined in SYMBOLIC_LIB.")


In [17]:
# Symbolic_KANLayer 사용 예제
import torch

# Symbolic_KANLayer 클래스 정의 (위에서 제공한 코드 사용)

if __name__ == "__main__":
    # Symbolic_KANLayer 초기화
    symbolic_layer = Symbolic_KANLayer(in_dim=3, out_dim=2, device='cpu')

    # 기호적 함수 할당 예제
    # 레이어 0, 뉴런 0: sin 함수
    symbolic_layer.fix_symbolic(j=0, i=0, fun_name='sin')
    # 레이어 0, 뉴런 1: cos 함수
    symbolic_layer.fix_symbolic(j=0, i=1, fun_name='cos')
    # 레이어 0, 뉴런 2: exp 함수
    symbolic_layer.fix_symbolic(j=0, i=2, fun_name='exp')

    # 입력 데이터 생성
    x = torch.randn(5, 3)  # (batch_size, in_dim)

    # 순전파 실행
    y, postacts = symbolic_layer(x)

    print("출력 y:")
    print(y)
    print("활성화 함수 후의 값 postacts:")
    print(postacts)


출력 y:
tensor([[4.6168, 0.0000],
        [1.5340, 0.0000],
        [2.2689, 0.0000],
        [4.5682, 0.0000],
        [2.2214, 0.0000]])
활성화 함수 후의 값 postacts:
tensor([[[ 0.7525,  0.7819,  3.0824],
         [ 0.0000,  0.0000,  0.0000]],

        [[-0.9460,  1.0000,  1.4800],
         [ 0.0000,  0.0000,  0.0000]],

        [[-0.1148,  0.2095,  2.1742],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.7138,  0.6708,  3.1836],
         [ 0.0000,  0.0000,  0.0000]],

        [[-0.2315,  0.9824,  1.4705],
         [ 0.0000,  0.0000,  0.0000]]])


### 6. KAN Class

In [ ]:

class KAN(nn.Module):
    def __init__(self, width, grid=5, k=3, base_fun='silu',
                 symbolic_enabled=True, affine_trainable=True,  # affine_trainable=True로 변경
                 device='cpu'):
        super(KAN, self).__init__()
        self.device = device
        self.depth = len(width) - 1
        self.width = width
        self.symbolic_enabled = symbolic_enabled

        self.act_fun = nn.ModuleList()
        self.symbolic_fun = nn.ModuleList()
        for l in range(self.depth):
            kan_layer = KANLayer(
                in_dim=width[l],
                out_dim=width[l + 1],
                num=grid,
                k=k,
                base_fun=torch.nn.SiLU(),
                device=device
            )
            self.act_fun.append(kan_layer)

            symbolic_layer = Symbolic_KANLayer(
                in_dim=width[l],
                out_dim=width[l + 1],
                device=device
            )
            self.symbolic_fun.append(symbolic_layer)

        self.node_bias = nn.ParameterList()
        self.node_scale = nn.ParameterList()
        self.subnode_bias = nn.ParameterList()
        self.subnode_scale = nn.ParameterList()
        for l in range(self.depth):
            self.node_bias.append(nn.Parameter(torch.zeros(width[l + 1], device=device), requires_grad=affine_trainable))
            self.node_scale.append(nn.Parameter(torch.ones(width[l + 1], device=device), requires_grad=affine_trainable))
            self.subnode_bias.append(nn.Parameter(torch.zeros(width[l + 1], device=device), requires_grad=affine_trainable))
            self.subnode_scale.append(nn.Parameter(torch.ones(width[l + 1], device=device), requires_grad=affine_trainable))

        self.to(device)

    def forward(self, x):
        for l in range(self.depth):
            x_numerical, preacts, postacts_numerical, postspline = self.act_fun[l](x)
            if self.symbolic_enabled:
                x_symbolic, postacts_symbolic = self.symbolic_fun[l](x)
            else:
                x_symbolic = torch.zeros_like(x_numerical)

            x = x_numerical + x_symbolic

            x = self.subnode_scale[l] * x + self.subnode_bias[l]
            x = self.node_scale[l] * x + self.node_bias[l]
        return x

    def symbolic_formula(self):
        formulas = []
        variables = [f"x{i}" for i in range(self.width[0])]
        for l in range(self.depth):
            layer_formulas = []
            for j in range(self.width[l + 1]):
                formula = ""
                for i in range(self.width[l]):
                    if self.symbolic_fun[l].mask.data[j, i] > 0 and self.symbolic_fun[l].funs[j][i] is not None:
                        a, b, c, d = self.symbolic_fun[l].affine.data[j, i]
                        func_name = self.symbolic_fun[l].funs_name[j][i]
                        var = variables[i]
                        term = f"{c.item():.3f} * {func_name}({a.item():.3f} * {var} + {b.item():.3f}) + "
                        formula += term
                formula = formula.rstrip(" + ")
                if formula == "":
                    formula = "0"
                layer_formulas.append(formula)
            variables = [f"h{l+1}_{j}" for j in range(self.width[l + 1])]
            formulas.append(layer_formulas)
        return formulas

    def fit(self, dataset, optimizer='Adam', steps=100, lr=0.001, lamb=0.0):
        '''
        KAN 모델을 학습합니다.

        Args:
        -----
            dataset : dict
                'train_input' 및 'train_label'을 포함하는 딕셔너리.
            optimizer : str
                사용할 옵티마이저 ('Adam' 또는 'LBFGS').
            steps : int
                학습 단계 수.
            lr : float
                학습률.
            lamb : float
                정규화 파라미터.

        Returns:
        --------
            None
        '''
        loss_fn = nn.MSELoss()

        if optimizer == 'Adam':
            optim = torch.optim.Adam(self.parameters(), lr=lr)
        elif optimizer == 'LBFGS':
            optim = torch.optim.LBFGS(self.parameters(), lr=lr)
        else:
            raise ValueError("Optimizer not recognized.")

        x_train = dataset['train_input']
        y_train = dataset['train_label']

        for step in range(steps):
            def closure():
                optim.zero_grad()
                output = self.forward(x_train)
                loss = loss_fn(output, y_train)
                # 필요 시 정규화 추가
                loss.backward()
                return loss

            if optimizer == 'LBFGS':
                optim.step(closure)
            else:
                loss = closure()
                optim.step()

            if step % 10 == 0:
                print(f"Step {step}: Loss = {loss.item()}")

    def prune(self, threshold=0.01):
        '''
        모델의 복잡도를 줄이기 위해 작은 계수를 가지는 연결을 제거합니다.

        Args:
        -----
            threshold : float
                가지치기 임계값. 이 값 이하의 절대값을 가진 계수는 제거됩니다.

        Returns:
        --------
            None
        '''
        for l in range(self.depth):
            # 수치적 층에서 작은 계수를 가지는 연결 제거
            small_sp = self.act_fun[l].scale_sp.abs() < threshold
            self.act_fun[l].mask.data[small_sp] = 0.0

            # 기호적 층에서 작은 계수를 가지는 연결 제거 (c 파라미터 기준)
            small_c = self.symbolic_fun[l].affine[:, :, 2].abs() < threshold
            self.symbolic_fun[l].mask.data[small_c] = 0.0

    def assign_symbolic_functions(self, assignments):
        '''
        여러 뉴런에 기호적 함수를 일괄 할당합니다.

        Args:
        -----
            assignments : list of tuples
                각 튜플은 (layer_index, output_neuron, input_neuron, function_name)
        '''
        for assignment in assignments:
            l, j, i, func_name = assignment
            if l < self.depth:
                self.symbolic_fun[l].fix_symbolic(j, i, func_name)
            else:
                print(f"Layer index {l} is out of range.")


In [12]:
# KAN 모델 사용 예시

# 네트워크 아키텍처 정의
width = [3, 5, 2]  # 입력 차원 3, 은닉층 5개 뉴런, 출력 차원 2

# KAN 모델 초기화
model = KAN(width=width, grid=5, k=3, device='cpu')

# 입력 데이터 생성
x = torch.randn(10, 3)

# 순전파 실행
output = model(x)

print("KAN 모델 출력:")
print(output)


KAN 모델 출력:
tensor([[-0.0638,  0.0495],
        [-0.0439, -0.0179],
        [ 0.2892,  0.1470],
        [-0.0409, -0.0491],
        [ 0.0861, -0.0115],
        [ 0.1815, -0.0007],
        [ 0.2272,  0.1203],
        [-0.0292,  0.0007],
        [ 0.0957,  0.0361],
        [-0.0505, -0.0630]], grad_fn=<AddBackward0>)


In [13]:
# 모델 학습 예제
import torch

# 네트워크 아키텍처 정의
width = [2, 5, 1]

# KAN 모델 초기화
model = KAN(width=width, grid=5, k=3, device='cpu')

# 데이터셋 생성
x_train = torch.rand(100, 2)
y_train = torch.sin(x_train[:, 0]) + x_train[:, 1] ** 2
y_train = y_train.unsqueeze(1)

dataset = {
    'train_input': x_train,
    'train_label': y_train
}

# 모델 학습
model.fit(dataset, optimizer='Adam', steps=100, lr=0.01)

# 테스트 데이터 생성
x_test = torch.rand(20, 2)
y_test = torch.sin(x_test[:, 0]) + x_test[:, 1] ** 2
y_test = y_test.unsqueeze(1)

# 예측 및 손실 계산
y_pred = model(x_test)
test_loss = nn.MSELoss()(y_pred, y_test)
print(f"Test Loss: {test_loss.item()}")


Step 0: Loss = 0.6423020362854004
Step 10: Loss = 0.1253228336572647
Step 20: Loss = 0.08409864455461502
Step 30: Loss = 0.014542357996106148
Step 40: Loss = 0.010408625937998295
Step 50: Loss = 0.0030659902840852737
Step 60: Loss = 0.0021975263953208923
Step 70: Loss = 0.0007675166707485914
Step 80: Loss = 0.00045903402497060597
Step 90: Loss = 0.0002122166333720088
Test Loss: 6.386656605172902e-05
